### Description

In [1]:
### This code aims to classify the survival classification for titanic members

### Imports

In [2]:
import pandas as pd

import numpy as np
from datetime import datetime
import xlsxwriter
import string
import os
import re
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller
import statsmodels.api as sm
from scipy import stats
from numpy.random import seed
from numpy.random import randn
from scipy.stats import shapiro
from sklearn.neighbors import KernelDensity
import seaborn as sns
import cvxpy as cp
from scipy.optimize import minimize

In [3]:
from scipy.optimize import minimize

## Specify parameters [1]

In [4]:


test_data = r"C:\Users\hugo.lu\repos\repos_backup\Kaggles\1. Titanic\test.csv"
train_data = r"C:\Users\hugo.lu\repos\repos_backup\Kaggles\1. Titanic\train.csv"



## Specify data

In [5]:
test_data = pd.read_csv(filepath_or_buffer = test_data, index_col = 0)

In [6]:
train_data = pd.read_csv(filepath_or_buffer = train_data, index_col = 0)

In [201]:
test_data.head(5)
test_data = test_data.reset_index()

In [200]:
train_data.head(5)
train_data=train_data.reset_index()

## State thesis and clean data

In [266]:
#Thesis: survivors depend on pclass,sex, age, SibSp, Cabin type, and possibly embarkation
def make_bool(df,cols):
    for item in cols:
        vals = list(df[item].drop_duplicates())
        #col = list(row.columns)[0]
        for i,j in enumerate(vals):
            df.at[(df[item] ==j) ,item] = i
    return df

## Based on the data, having cabin as NA also seems to be a good divisor of socioeconomic class.

## Therefore this is added as a new variable instead of cabin number. Looking at cabin number seems to provide no indication
## of anything really

In [267]:
x_vars = []
for item in list(train_data.columns):
    x_vars.append(item)
bool_converts =["Survived", "Pclass", "Sex", "Cabin","Embarked"]
train_data_copy = train_data.copy(deep = True)



In [268]:
train_data_copy = make_bool(train_data_copy,bool_converts) 

In [269]:
def remove_nans(df,col,how_to):
    if how_to == "kde":
        no_nans = df[col].dropna()
        num_nans = df[col].isnull().sum()
        print(num_nans)
        x = np.array(no_nans)
        xs = np.linspace(min(no_nans),max(no_nans),1000)
        # expects n x p matrix with p features
        x.shape = (len(x), 1)
        xs.shape = (len(xs), 1)
        #### Question@ is this the correct KDE to use? Probably can be refined and generalised....
        kde = KernelDensity(bandwidth=((max(no_nans)- min(no_nans))/30)).fit(x)
        dens = np.exp(kde.score_samples(xs))
        #plt.plot(xs, dens)
        probs = dens / sum(dens)
        xs_comma = np.reshape(xs,newshape = (len(xs),))
        mc = np.random.choice(xs_comma, size=num_nans, replace=True, p=probs)
        print(len(mc))
        df.loc[df[col].isnull(), col] = list(mc)
        return df
    else: 
        print("other")



In [270]:
train_data_copy = remove_nans(train_data_copy,"Age","kde")
train_data_copy['Cabin_edit'] = train_data_copy['Cabin'].apply(lambda x: 1 if np.isnan(x)else 0)

177
177


In [271]:
train_data_copy.head(2)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Cabin_edit
0,1,0,0,"Braund, Mr. Owen Harris",0,22.0,1,0,A/5 21171,7.2500,NaN,0,1
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,38.0,1,0,PC 17599,71.2833,1,1,0


In [272]:
x_vars.remove("Name")
x_vars.remove("Ticket")
x_vars.remove("Cabin")
x_vars.append("Cabin_edit")
x_vars.remove("PassengerId")
x_vars.remove("Survived")

In [275]:
x_vars

['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked', 'Cabin_edit']

In [276]:
y_vars = ['Survived']

In [278]:
train_data_copy = train_data_copy[x_vars + y_vars]

In [284]:
train_data_copy.isna().sum()

Pclass        0
Sex           0
Age           0
SibSp         0
Parch         0
Fare          0
Embarked      2
Cabin_edit    0
Survived      0
dtype: int64

In [285]:
train_data_copy  = train_data_copy.dropna()

### Train models

In [296]:
from sklearn import model_selection 
from sklearn.ensemble import BaggingClassifier 
from sklearn.tree import DecisionTreeClassifier 
import pandas as pd 
  
# load the data 

X = np.array(train_data_copy[x_vars])
Y = np.array(train_data_copy[y_vars])
#Y = np.reshape(Y,newshape = (len(Y),1))
  
seed = 8
kfold = model_selection.KFold(n_splits = 10, 
                       random_state = seed) 
  
# initialize the base classifier 
base_cls = DecisionTreeClassifier(min_samples_split = 10) 
  
# no. of base classifier 
num_trees = 700
  
# bagging classifier 
model = BaggingClassifier(base_estimator = base_cls, 
                          n_estimators = num_trees, 
                          random_state = seed,
                         max_samples = round(len(train_data_copy)/1.3),
                         bootstrap = True)
fit = model.fit(X,Y)
  
results = model_selection.cross_val_score(model, X, Y, cv = kfold) 
print("accuracy :") 
print(results.mean()) 

C:\Users\hugo.lu\Anaconda3\lib\site-packages\sklearn\ensemble\bagging.py:622: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\hugo.lu\Anaconda3\lib\site-packages\sklearn\ensemble\bagging.py:622: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\hugo.lu\Anaconda3\lib\site-packages\sklearn\ensemble\bagging.py:622: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\hugo.lu\Anaconda3\lib\site-packages\sklearn\ensemble\bagging.py:622: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samp

accuracy :
0.8155771195097037


In [303]:
predictions_train = ((fit.predict(X)))
output = pd.DataFrame()
output['y_hat'] = predictions_train
output['y'] = Y
output['true_positives'] = None
output.at[(output['y'] ==1) & (output['y_hat']==1),'true_positives'] = 1
output['true_negatives'] = None
output.at[(output['y'] ==0) & (output['y_hat']==0),'true_negatives'] = 1
output['false_positives'] = None
output.at[(output['y'] ==0) & (output['y_hat']==1),'false_positives'] = 1
output['false_negatives'] = None
output.at[(output['y'] ==1) & (output['y_hat']==0),'false_negatives'] = 1

In [304]:
output.sum()

y_hat              300.0
y                  340.0
true_positives     276.0
true_negatives     525.0
false_positives     24.0
false_negatives     64.0
dtype: float64

In [300]:
output.sum()
print("the true positive rate is:"+str(output['true_positives'].sum()/output['y_hat'].sum()))

the true positive rate is:0.92


In [307]:
print("the accuracy rate on the train is:"+str((output['true_negatives'].sum()+output['true_positives'].sum())/len(output)))

the accuracy rate on the train is:0.9010123734533183


## Testing the model

#### Cleaning the test set

In [323]:
test_data_copy = test_data.copy(deep = True)
test_data_copy['Survived'] = None
test_data_copy = test_data_copy.reset_index()

In [324]:
test_data_copy = make_bool(test_data_copy,bool_converts) 

In [325]:
test_data_copy = remove_nans(test_data_copy,"Age","kde")
test_data_copy['Cabin_edit'] = test_data_copy['Cabin'].apply(lambda x: 1 if np.isnan(x)else 0)

86
86


In [326]:
test_data_copy = test_data_copy[x_vars+y_vars]

In [327]:
test_data_copy.isna().sum()

Pclass          0
Sex             0
Age             0
SibSp           0
Parch           0
Fare            1
Embarked        0
Cabin_edit      0
Survived      418
dtype: int64

In [328]:
test_data_copy.at[test_data_copy['Fare'].isna(),'Fare'] = test_data_copy['Fare'].mean()

In [329]:
test_data_copy.isna().sum()

Pclass          0
Sex             0
Age             0
SibSp           0
Parch           0
Fare            0
Embarked        0
Cabin_edit      0
Survived      418
dtype: int64

In [330]:
X_test = test_data_copy[x_vars]

In [332]:
predictions_test = ((fit.predict(X_test)))
output = pd.DataFrame()
test_data_copy['Survived'] = predictions_test
#output['y'] = Y


In [335]:
train_data_copy['Survived'].sum()/len(train_data_copy)

0.38245219347581555